## The Game of 24
One of my favorite card game growing up. Per Wikipedia:

```
    The 24 puzzle is an arithmetical puzzle in which the objective is to find a way to manipulate four integers so that the end result is 24. For example, for the numbers 4, 7, 8, 8, a possible solution is (7 - (8 / 8)) * 4. Note that all four numbers must be used exactly once
```

In [1]:
%load_ext autoreload
%autoreload 2

### Game Definition
In the `game.py` file, I have setup the `Game` interface

In [17]:
from twenty_four.game import Game

game = Game(init_state=[4, 7, 8, 8])

One can apply operations to the game state using the indices of the number in the current state.
Also, the `Game` class supports **undo** operation

In [18]:
print(f"Current state: {game.get_cur_state()}")
game.divide(1, 3)
print(f"Current state: {game.get_cur_state()}")
game.undo()
print(f"Current state: {game.get_cur_state()}")
game.divide(2, 3)
print(f"Current state: {game.get_cur_state()}")
game.subtract(1, 2)
print(f"Current state: {game.get_cur_state()}")
game.multiply(0, 1)
print(f"Current state: {game.get_cur_state()}")

Current state: [4.0, 7.0, 8.0, 8.0]
Current state: [4.0, 8.0, 0.875]
Current state: [4.0, 7.0, 8.0, 8.0]
Current state: [4.0, 7.0, 1.0]
Current state: [4.0, 6.0]
Current state: [24.0]


The solution can be printed as following

In [15]:
print(game.get_solution_expr())

4 * (7 - 8 / 8)


### Solver
In `basic_solver.py` and `agent_solver.py`, three `Solver` classes are implemented.

- `BruteForceSolver`: recursively explore next possible operation. Backtrack if not solvable
- `SmartSolver`: recursively explore next possible operation and cache next step that leads to a solution. The class can also play random games to build the memorization cache
- `AgentSolver`: agentic solver by giving LLM with the following tools:
  - `perform_op`: perform an operation on the current game state.
  - `undo_op`: undo the last operation.
  - `direct_solve`: solve using brute force when there's only two numbers in the state.


### Comparison
In the following cells, we tested the performance of each solver using a few representative examples

In [2]:
from time import perf_counter
from twenty_four.basic_solver import Solver, BruteForceSolver, SmartSolver
from twenty_four.agent_solver import AgentSolver


In [19]:
def benchmark(solver: Solver, games: list[list[float]]) -> list[float]:
    """Run the solver on a list of games, print the solution and compute the average duration of games."""
    durs = []
    for state in games:
        game = Game(init_state=state)
        start = perf_counter()
        solution = solver.solve(game)
        durs.append(perf_counter() - start)
        print(f"{state} -> {solution} ({durs[-1]:.10f}s)")
    print(f"Average duration: {sum(durs) / len(durs):.10f}s")


In [20]:
example_games = [
    [1, 2, 3, 4],
    [7, 3, 3, 7],
    [1, 1, 1, 1],
    [3, 8, 3, 8],
    [11, 1, 11, 5],
    [13, 1, 13, 7],
    [1, 3, 8, 4],
]

#### Brute-Force Solver

In [5]:
brute_force_solver = BruteForceSolver()
benchmark(brute_force_solver, example_games)

[1, 2, 3, 4] -> 4 * (3 + 1 + 2) (0.0002763750s)
[7, 3, 3, 7] -> 7 * (3 + 3 / 7) (0.0020739580s)
[1, 1, 1, 1] -> None (0.0090940000s)
[3, 8, 3, 8] -> 8 / (3 - 8 / 3) (0.0017260000s)
[11, 1, 11, 5] -> (11 * 11 - 1) / 5 (0.0023009580s)
[13, 1, 13, 7] -> (13 * 13 - 1) / 7 (0.0023006670s)
[1, 3, 8, 4] -> 8 + 4 * (1 + 3) (0.0002533750s)
Average duration: 0.0025750476s


#### Smart Solver with Memorization

In [9]:
smart_solver = SmartSolver(num_games=1000)
benchmark(smart_solver, example_games)

======Training Started======
Solved 1 games
Solved 101 games
Solved 201 games
Solved 301 games
Solved 401 games
Solved 501 games
Solved 601 games
Solved 701 games
Solved 801 games
Solved 901 games
Solved 1001 games
========Training Done========
[1, 2, 3, 4] -> 4 * (3 + 1 + 2) (0.0000175830s)
[7, 3, 3, 7] -> 7 * (3 + 3 / 7) (0.0000142920s)
[1, 1, 1, 1] -> None (0.0001004170s)
[3, 8, 3, 8] -> 8 / (3 - 8 / 3) (0.0000145830s)
[11, 1, 11, 5] -> (11 * 11 - 1) / 5 (0.0000143750s)
[13, 1, 13, 7] -> (13 * 13 - 1) / 7 (0.0000132910s)
[1, 3, 8, 4] -> 8 + 4 * (1 + 3) (0.0000132080s)
Average duration: 0.0000268213s


#### Agentic Solver using LangGraph and OpenAI models

gpt-4o model

In [21]:
agent_solver = AgentSolver(model="gpt-4o")
benchmark(agent_solver, example_games)

[1, 2, 3, 4] -> 2 * 3 * 1 * 4 (4.0696190410s)
[7, 3, 3, 7] -> None (55.3556951660s)
[1, 1, 1, 1] -> None (1.8170785420s)
[3, 8, 3, 8] -> None (21.8309424590s)
[11, 1, 11, 5] -> None (43.2660496670s)
[13, 1, 13, 7] -> None (9.2481402910s)
[1, 3, 8, 4] -> None (6.0013270000s)
Average duration: 20.2269788809s


o4-mini model

In [7]:
agent_solver = AgentSolver(model="o4-mini")
benchmark(agent_solver, example_games)

[1, 2, 3, 4] -> 1 * 4 * 2 * 3 (16.9661647920s)
[7, 3, 3, 7] -> None (103.9218697500s)
[1, 1, 1, 1] -> None (3.3071891660s)
[3, 8, 3, 8] -> 8 / (3 - 8 / 3) (49.0721780000s)
[11, 1, 11, 5] -> (11 * 11 - 1) / 5 (87.4103180840s)
[13, 1, 13, 7] -> None (26.6267680000s)
[1, 3, 8, 4] -> (8 + 4) * (3 - 1) (18.2180870840s)
Average duration: 43.6460821251s


o3 model

In [11]:
agent_solver = AgentSolver(model="o3")
benchmark(agent_solver, example_games)

[1, 2, 3, 4] -> 1 * 4 * 2 * 3 (25.7439446250s)
[7, 3, 3, 7] -> 7 * (3 + 3 / 7) (57.1472106670s)
[1, 1, 1, 1] -> None (32.1572789170s)
[3, 8, 3, 8] -> 8 / (3 - 8 / 3) (93.9317958340s)
[11, 1, 11, 5] -> (11 * 11 - 1) / 5 (228.6280841660s)
[13, 1, 13, 7] -> (13 * 13 - 1) / 7 (122.4673297500s)
[1, 3, 8, 4] -> (8 + 4) * (3 - 1) (15.2604533330s)
Average duration: 82.1908710417s
